In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, recall_score, f1_score, multilabel_confusion_matrix
from imblearn.metrics import geometric_mean_score, specificity_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingClassifier
import json
import pandas as pd
import numpy as np
import sys, os
import warnings
import datetime
#import pickle
from joblib import dump, load
from pathlib import Path

from sklearn import metrics
from sklearn.metrics import classification_report

from art.attacks.evasion import FastGradientMethod, DecisionTreeAttack, ZooAttack
from art.estimators.classification import SklearnClassifier
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/estimators/certification/__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [3]:


XCols = ['mean', 'sum', 'std']
label = 'label'



In [4]:
#df = pd.read_csv(sys.argv[1], dtype = fieldTypes, parse_dates = ['time_start'])
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/stats_nov_apr.csv", parse_dates = ['time'])
df.fillna(0, inplace=True)

weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]




In [5]:
# trainCon = df['time'] >= pd.Timestamp('2020-03-01')
#trainCon = df['time'] < pd.Timestamp('2020-01-01')
#trainCon = ((df['time'] >= pd.Timestamp('2020-01-01')) & (df['time'] < pd.Timestamp('2020-03-01')))
trainCon = df['time'].dt.isocalendar().week.isin(weeks[0])
#trainCon = df['time'].dt.week.isin([44,45])


#testCon = df['time'] > pd.Timestamp('2020-02-29')
#testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1W').tolist()
#testCons = pd.date_range('2019-11-01', '2019-12-01', freq = '1W').tolist()

#X = dfTrain.iloc[:, 2:5]
#y = dfTrain.iloc[:, 1]
X = df[trainCon][XCols]
y = df[trainCon][label]

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


In [7]:
#clf = RandomForestClassifier(n_estimators=20, n_jobs=50)
#rfc = RandomForestClassifier(n_jobs=18)
#svc = SVC(verbose=True)
dtc = DecisionTreeClassifier()
#knn = KNeighborsClassifier(n_jobs=54)
#mv = VotingClassifier(estimators=[('knn', knn), ('dt', dtc), ('rf', rfc), ('svm',svc)], voting='hard', n_jobs=54)

modelFile = Path(f"model/dtc_0_1sec")
if modelFile.is_file():
    print(f"loading model {modelFile}")
    dtc = load(modelFile)
else:
    print(f"start training for weeks {weeks[2]}")
    dtc.fit(X_train.values, y_train)
    print("DTC trained")
    #dump(rfc, f"model/rfc_{sys.argv[2]}")
    dump(dtc, modelFile)


loading model model/dtc_0_1sec


In [21]:
for date in testCons:
    testCon = df['time'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[XCols]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    
    #y_pred = clf.predict(X)
    #y_rfc = rfc.predict(X)
    #print("rfc predict")
    #y_svc = svc.predict(X)
    #print("svc predict")
    y_dtc = dtc.predict(X)
    #print("dtc predict")
    #y_knn = knn.predict(X)
    #print("knn predict")
    #y_mv = mv.predict(X)
    #print("mv predict")
    
    print(f"{date.week}", metrics.f1_score(y, y_dtc, average='micro'))
    print(classification_report(y,y_dtc))#, labels=devs))

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


44 0.716661276583152


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

              precision    recall  f1-score   support

           7       0.83      0.49      0.62      6745
           8       0.51      0.91      0.65     36559
           9       0.36      0.02      0.04      9418
          10       0.90      0.91      0.91     12953
          12       0.83      0.38      0.52      7509
          14       0.84      0.82      0.83     11626
          15       0.63      0.50      0.56     23081
          16       0.53      0.49      0.51     66903
          17       0.00      0.00      0.00         0
          18       0.90      0.97      0.93     66743
          19       0.61      0.07      0.12     36295
          20       0.69      0.60      0.64       944
          22       0.87      1.00      0.93     81312
          23       0.51      0.50      0.50      4289
          24       0.72      0.19      0.31     11994
          25       0.84      0.99      0.91     16473
          27       0.98      0.33      0.49      5368
          28       0.47    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


...

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/validation.py:623: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():


17 0.6363752677341229


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

              precision    recall  f1-score   support

           7       0.62      0.29      0.40     24604
           8       0.00      0.00      0.00         0
           9       0.31      0.16      0.21     64662
          10       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          14       0.78      0.64      0.70     67195
          15       0.07      0.29      0.12     18700
          16       0.05      0.81      0.10     14281
          17       0.78      0.34      0.48     23117
          18       0.99      0.97      0.98    403245
          19       1.00      0.94      0.97    220232
          20       0.02      0.00      0.00    145608
          22       0.00      0.00      0.00         0
          23       0.52      0.23      0.32     51621
          24       0.55      0.15      0.23     74235
          25       0.56      0.95      0.71     25054
          27       0.95      0.45      0.61     35077
          28       0.98    

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
advClassifier = ScikitlearnDecisionTreeClassifier(model=dtc)
dtAdv = DecisionTreeAttack(classifier=advClassifier)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [22]:
for date in testCons:
    testCon = df['time'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[XCols]
    y = testDF[label]
    
    if len(X) == 0: 
        continue
    x_test_adv = dtAdv.generate(x=X.values)
   
    np.savetxt(f"dtAdv-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    y_dtc = advClassifier.predict(x_test_adv)
    print("DT test")
    print(f"{date.week}", metrics.f1_score(y, np.rint(y_dtc).argmax(axis=1), average='micro'))
    print(classification_report(y, np.rint(y_dtc).argmax(axis=1)))#, labels=devs))
  

Decision tree attack: 100%|██████████| 473091/473091 [03:27<00:00, 2281.14it/s]


DT test
44 0.0


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0
           7       0.00      0.00      0.00    6745.0
           8       0.00      0.00      0.00   36559.0
           9       0.00      0.00      0.00    9418.0
          10       0.00      0.00      0.00   12953.0
          12       0.00      0.00      0.00    7509.0
          13       0.00      0.00      0.00       0.0
          14       0.00      0.00      0.00   11626.0
          15       0.00      0.00      0.00   23081.0
          16       0.00      0.00      0.00   66903.0
          17       0.00      0.00      0.00       0.0
          18       0.00      0.00      0.00   66743.0
          19       0.00      0.00      0.00   36295.0
          20       0.00      0.00      0.00     944.0
          22       0.00      0.00      0.00   81312.0
          23       0.00      0.00      0.00    4289.0
          24       0.00    

Decision tree attack: 100%|██████████| 2483988/2483988 [19:17<00:00, 2145.78it/s]


...

DT test
49 0.0


/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_pr

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0
           7       0.00      0.00      0.00   41723.0
           8       0.00      0.00      0.00  220605.0
           9       0.00      0.00      0.00   75860.0
          10       0.00      0.00      0.00   69738.0
          12       0.00      0.00      0.00   54084.0
          13       0.00      0.00      0.00       0.0
          14       0.00      0.00      0.00   64200.0
          15       0.00      0.00      0.00  115643.0
          16       0.00      0.00      0.00  717815.0
          17       0.00      0.00      0.00   24382.0
          18       0.00      0.00      0.00  404703.0
          19       0.00      0.00      0.00  220202.0
          20       0.00      0.00      0.00    6236.0
          22       0.00      0.00      0.00  597306.0
          23       0.00      0.00      0.00   24102.0
          24       0.00    

Decision tree attack:  14%|█▍        | 550596/3891198 [03:54<23:42, 2347.74it/s]


KeyboardInterrupt: 

In [21]:
print(multilabel_confusion_matrix(y, np.rint(y_dtc).argmax(axis=1)))

[[[361443. 111648.]
  [     0.      0.]]

 [[195288. 277803.]
  [     0.      0.]]

 [[414958.  51388.]
  [  6745.      0.]]

 [[436532.      0.]
  [ 36559.      0.]]

 [[460280.   3393.]
  [  9418.      0.]]

 [[460138.      0.]
  [ 12953.      0.]]

 [[465582.      0.]
  [  7509.      0.]]

 [[456485.  16606.]
  [     0.      0.]]

 [[461465.      0.]
  [ 11626.      0.]]

 [[450010.      0.]
  [ 23081.      0.]]

 [[406188.      0.]
  [ 66903.      0.]]

 [[460838.  12253.]
  [     0.      0.]]

 [[406348.      0.]
  [ 66743.      0.]]

 [[436796.      0.]
  [ 36295.      0.]]

 [[472147.      0.]
  [   944.      0.]]

 [[391779.      0.]
  [ 81312.      0.]]

 [[468802.      0.]
  [  4289.      0.]]

 [[461097.      0.]
  [ 11994.      0.]]

 [[456618.      0.]
  [ 16473.      0.]]

 [[467723.      0.]
  [  5368.      0.]]

 [[443724.      0.]
  [ 29367.      0.]]

 [[468093.      0.]
  [  4998.      0.]]

 [[461974.      0.]
  [ 11117.      0.]]

 [[451614.      0.]
  [ 21477.    

In [8]:
from art.metrics import RobustnessVerificationTreeModelsCliqueMethod
aclassifier = SklearnClassifier(model=dtc)
rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=aclassifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


AttributeError: 'ScikitlearnDecisionTreeClassifier' object has no attribute 'get_trees'

In [8]:
advClassifier = SklearnClassifier(dtc)
zoo = DecisionTreeAttack(classifier=advClassifier)

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
dist = 0
for date in testCons:
    path = Path(f"dt1sec_0adv-{date.week}.csv")
    if path.is_file():
        print("test exists")
        x_test_adv = np.loadtxt(f"dt1sec_0adv-{date.week}.csv", delimiter=",", dtype=str)
    testCon = df['time'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    testDF = df[testCon]

    X = testDF[XCols]
    y = testDF[label]
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
    
    if len(X) == 0: 
        continue
    #x_test_adv = zoo.generate(x=X_test.values, y=y_test-2)
   
    #np.savetxt(f"RFzoo2-{date.week}.csv", x_test_adv, delimiter=",")

   # Step 7: Evaluate the ART classifier on adversarial test examples
    #y_rfc = advClassifier.predict(x_test_adv)
    for index, sample in enumerate (x_test_adv):
        dist = dist + np.linalg.norm(sample-X[index])
    print(dist)
    #print(dtc.score(x_test_adv, y_test))
    #print(f"{date.week}", metrics.f1_score(y, y_rfc, average='micro'))

test exists


KeyError: 0

In [ ]:
advClassifier = ScikitlearnDecisionTreeClassifier(model=dtc)
dtAdv = Zero(classifier=advClassifier)